## Visit computation
The purpose of this notebook is to compare the mobility of users who live in the "treatment" (ZATs near new cable car) and "control" zones (similar ZATs but no new cable car). We will map the visits we computed in the `mobility_analysis` notebook to the treatment and control group information and point of interest (POIs) computed from google places.

In [ ]:
import os
from dotenv import load_dotenv

load_dotenv()

working_dir = os.getenv("WORKING_DIR")
os.environ["USE_PYGEOS"] = "0"

from setup import *
from plotting import *
from preprocess import *
from analysis import *

import pandas as pd
import numpy as np
import skmob

c = read_config(f"{working_dir}configs/config_2019.yml")
(
    year,
    datatypes,
    initial_cols,
    sel_cols,
    final_cols,
    minlon,
    maxlon,
    minlat,
    maxlat,
) = get_config_vars(c=c, mode="preprocess")
min_days, min_pings = get_config_vars(c=c, mode="user_qc")

where = get_dirs(working_dir, year=year, min_days=min_days, min_pings=min_pings)

meta_dir = where.meta_dir

hw_dir = f"{where.pass_qc_dir}home_work_locs/home/"
visit_dir = f"{hw_dir}in_zats/visits/"
out_dir = f"{visit_dir}w_pois/"

for d in [visit_dir, out_dir]:
    ensure_directory_exists(d)

sel_zat_home_locs_meta_fp = f"{hw_dir}selected_txt_control_home_locs_w_zats_for_users_pass_qc_w_treatment_group.csv"

### Load home locations with ZATs and treatment group for users living in treatment and control ZATs

In [ ]:
sel_zat_home_locs_meta = pd.read_csv(sel_zat_home_locs_meta_fp)
sel_zat_home_locs_meta.head()

### Load shapefiles with the POI information 
There is one with a buffer of 15 meters around the google places POIs and one with 20 meters buffer.

In [ ]:
# for POIs with 15m Buffer
shp_name, gdf_from_shp = get_shp_to_assign_poi(
    shp_dir=f"{meta_dir}places/Buffer Shapefiles/", config=c, radius=15, plot=True
)

### Load visits and map the visits to the POIs
Load the visit locations for users living within the treatment and control ZATs. Note that these are rough estimates and may need to be recomputed. 

In [ ]:
visit_durations = [20, 30, 60]
min_minutes = 1e12  # or 1440

In [ ]:
# only execute this cell if you want to map and write user visits to POIs, not needed if you have already done this
for i in tqdm(
    range(0, len(visit_durations)),
    desc=f"Mapping and writing user visits to POIs from {shp_name}",
):
    stopping_time = visit_durations[i]
    visits_filename = f"users_living_in_sel_zat_visits_atleast_{stopping_time}min_nodatafor_{min_minutes}_minutes"  # modify fp as needed, e.g. add _nodatafor_1440_minutes'
    visits_fp = f"{visit_dir}{visits_filename}.csv"
    outfilename = f"{visits_filename}_w_poi_from_{shp_name}_shp"
    visit_df = read_visits(visits_fp, uid_treat_group_info=sel_zat_home_locs_meta)
    visits_w_poi_df = calc_write_visit_pois(
        visit_df=visit_df,
        regions_gdf=gdf_from_shp,
        cols_to_keep=(
            list(visit_df.columns)
            + [
                "name",
                "dup",
                "status",
                "category",
                "id",
                "BUFF_DIST",
                "ORIG_FID",
                "Shape_Area",
            ]
        ),
        out_dir=out_dir,
        subdir_name=shp_name,
        outfilename=outfilename,
    )

#### Some results (from 2019): 
**For the 15m Buffer**
- There were 3007082 visits in the file. 1157808 instances occured where visits mapped to named POIs (some visits may map to multiple POIs). Wrote data to users_living_in_sel_zat_visits_atleast_20min_w_poi_from_POI_Buffer15m_shp
- There were 2487677 visits in the file. 934178 instances occured where visits mapped to named POIs (some visits may map to multiple POIs). Wrote data to users_living_in_sel_zat_visits_atleast_30min_w_poi_from_POI_Buffer15m_shp
- There were 1861875 visits in the file. 643682 instances occured where visits mapped to named POIs (some visits may map to multiple POIs). Wrote data to users_living_in_sel_zat_visits_atleast_60min_w_poi_from_POI_Buffer15m_shp

**For the 20m Buffer**
- There were 3007082 visits in the file. 1967679 instances occured where visits mapped to named POIs (some visits may map to multiple POIs). Wrote data to users_living_in_sel_zat_visits_atleast_20min_w_poi_from_POI_Buffer20m_shp

- There were 2487677 visits in the file. 1583775 instances occured where visits mapped to named POIs (some visits may map to multiple POIs). Wrote data to users_living_in_sel_zat_visits_atleast_30min_w_poi_from_POI_Buffer20m_shp

- There were 1861875 visits in the file. 1089400 instances occured where visits mapped to named POIs (some visits may map to multiple POIs). Wrote data to users_living_in_sel_zat_visits_atleast_60min_w_poi_from_POI_Buffer20m_shp

#### Some logs for the files with the additional `_nodatafor_1440_minutes` filtering: 

**For the 15m Buffer**
- There were 2763819 visits in the file. 1080490 instances occured where visits mapped to named POIs (some visits may map to multiple POIs). Wrote data to users_living_in_sel_zat_visits_atleast_20min_nodatafor_1440_minutes_w_poi_from_POI_Buffer15m_shp

- There were 2242918 visits in the file. 856483 instances occured where visits mapped to named POIs (some visits may map to multiple POIs). Wrote data to users_living_in_sel_zat_visits_atleast_30min_nodatafor_1440_minutes_w_poi_from_POI_Buffer15m_shp

- There were 1613447 visits in the file. 564951 instances occured where visits mapped to named POIs (some visits may map to multiple POIs). Wrote data to users_living_in_sel_zat_visits_atleast_60min_nodatafor_1440_minutes_w_poi_from_POI_Buffer15m_shp

**For the 20m Buffer**
- There were 2763819 visits in the file. 1836594 instances occured where visits mapped to named POIs (some visits may map to multiple POIs). Wrote data to users_living_in_sel_zat_visits_atleast_20min_nodatafor_1440_minutes_w_poi_from_POI_Buffer20m_shp

- There were 2242918 visits in the file. 1452026 instances occured where visits mapped to named POIs (some visits may map to multiple POIs). Wrote data to users_living_in_sel_zat_visits_atleast_30min_nodatafor_1440_minutes_w_poi_from_POI_Buffer20m_shp

- There were 1613447 visits in the file. 955859 instances occured where visits mapped to named POIs (some visits may map to multiple POIs). Wrote data to users_living_in_sel_zat_visits_atleast_60min_nodatafor_1440_minutes_w_poi_from_POI_Buffer20m_shp
